In [1]:
%run arm_dynamics.ipynb

jit after {'jit': True, 'jit_options': {'flags': '-Ofast'}, 'compiler': 'shell'}
number of joints = 4
floating_base found
no floating_base found
floating_base found
no floating_base found
floating_base found
no floating_base found
floating_base found
no floating_base found
[115.143, 117.481, 3.33301, 177.52]
[115.143, 117.481, 3.33301, 177.52]
[3.31188, 3.45983, 2.63969, 60.0617]
[3.31188, 3.45983, 2.63969, 60.0617]


In [4]:
x_next, states, u, parameters = rig_dyn.forward_dynamics(gravity=9.81, floating_base_id=None, floating_base_bias_f=None, J_uv=None, coupled=False)
F_next_ = cs.Function('forward_dynamics_base', [states, u, parameters, ss.q_min, ss.q_max], [x_next])
F_next_.save("model.casadi")

floating_base found
no floating_base found
floating_base found
no floating_base found
floating_base found
no floating_base found
parameters used


In [3]:
q0 = [1.91318, 1.33339, 0.99245, 3.16073]
q0dot = [0.00751799, -0.0073626, 0.00281749, 0.0222385]
u0 = [0.0340454, -0.00441695, 0.0062739, -0.00505981]
dt0 = 0.04

fd_p = cs.vertcat(rigid_body_p_est, arm_Hyd__p_est, dt0, base_T0)
xt_1 = cs.vertcat(q0, q0dot)

xt = F_next_(xt_1, u0, fd_p,  joint_min, joint_max)
print(xt)

[1.91361, 1.5, 0.992564, 3.1607, 0.0113298, 0, 0, 0]


In [4]:
# # c , cpp or matlab code generation for forward dynamics
# F_next_d.generate("F_next_.c")
# os.system(f"gcc -fPIC -shared F_next_.c -o libMnext.so")